In [25]:
############################################# IMPORTING ################################################
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox as mess
import tkinter.simpledialog as tsd
import cv2, os
import csv
import numpy as np
from PIL import Image
import pandas as pd
import datetime
import time

############################################# FUNCTIONS ################################################
def assure_path_exists(path):
    dir = os.path.dirname(path)
    if not os.path.exists(dir):
        os.makedirs(dir)
##################################################################################
# 更新幾點幾分幾秒
def tick():
    time_string = time.strftime('%H:%M:%S')
    clock.config(text = time_string)
    clock.after(200, tick)

#######################################################################################
def TakeImages():
    columns = ['SERIAL NO.', '', 'ID', '', 'NAME']
    assure_path_exists("StudentDetails/")
    assure_path_exists("TrainingImage/")
    serial = 0
    exists = os.path.isfile("StudentDetails\StudentDetails.csv")
    if exists:
        with open("StudentDetails\StudentDetails.csv", 'r', encoding='utf-8') as csvFile1:
            reader1 = csv.reader(csvFile1)
            for l in reader1:
                serial = serial + 1
        serial = (serial // 2)
        csvFile1.close()
    else:
        with open("StudentDetails\StudentDetails.csv", 'a+', encoding='utf-8') as csvFile1:
            writer = csv.writer(csvFile1)
            writer.writerow(columns)
            serial = 1
        csvFile1.close()
    Id = (txt.get())
    name = (txt2.get().encode('utf-8'))
    if ((name.decode().isalpha()) or (' ' in name)):
        cam = cv2.VideoCapture(0)
        detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        sampleNum = 0
        while True:
            ret, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector.detectMultiScale(gray, 1.2, 5)
            for (x, y, w, h) in faces:
                cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
                # incrementing sample number
                sampleNum = sampleNum + 1
                # saving the captured face in the dataset folder TrainingImage
                
                cv2.imwrite("TrainingImage\ " + "." + str(serial) + "." + Id + '.' + str(sampleNum) + ".jpg",
                            gray[y:y + h, x:x + w])
            # display the frame
            cv2.imshow('Taking Images', img)
            
            # wait for 100 miliseconds
            if cv2.waitKey(200) == ord('q'):
                break
            # break if the sample number is morethan 10
            elif sampleNum > 10:
                break
        cam.release()
        cv2.destroyAllWindows()
        res = "Images Taken for ID : " + Id
        row = [serial, '', Id, '', name.decode()]
        with open('StudentDetails\StudentDetails.csv', 'a+', encoding='utf-8') as csvFile:
            writer = csv.writer(csvFile)
            writer.writerow(row)
        csvFile.close()
        message1.configure(text=res)
    else:
        if (name.isalpha() == False):
            res = "Enter Correct name"
            message.configure(text=res)

########################################################################################

def TrainImages():
    assure_path_exists("TrainingImageLabel/")
    recognizer = cv2.face_LBPHFaceRecognizer.create()
    detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces, ID = getImagesAndLabels("TrainingImage")
    print(ID)
    try:
        recognizer.train(faces, np.array(ID))
    except:
        mess._show(title='No Registrations', message='Please Register someone first!!!')
        return
    recognizer.save("TrainingImageLabel\Trainner.yml")
    res = "Profile Saved Successfully"
    message1.configure(text=res)
    message.configure(text='Total Registrations till now  : ' + str(max(ID)))

############################################################################################3

def getImagesAndLabels(path):
    # get the path of all the files in the folder
    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]
    # create empth face list
    faces = []
    # create empty ID list
    Ids = []
    # now looping through all the image paths and loading the Ids and the images
    for imagePath in imagePaths:
        # loading the image and converting it to gray scale
        pilImage = Image.open(imagePath).convert('L')
        # Now we are converting the PIL image into numpy array
        imageNp = np.array(pilImage, 'uint8')
        # getting the Id from the image
        ID = int(os.path.split(imagePath)[-1].split(".")[1])
        # extract the face from the training image sample
        faces.append(imageNp)
        Ids.append(ID)
    return faces, Ids

###########################################################################################

def TrackImages():
    assure_path_exists("Attendance/")
    assure_path_exists("StudentDetails/")
    flag = True
    msg = ''
    i = 0
    j = 0
    recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
    exists3 = os.path.isfile("TrainingImageLabel\Trainner.yml")
    if exists3:
        recognizer.read("TrainingImageLabel\Trainner.yml")
    else:
        mess._show(title='Data Missing', message='Please click on Save Profile to reset data!!')
        return
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath);

    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    col_names = ['Id', '', 'Name', '', 'Date', '', 'Time', '', 'State']
    exists1 = os.path.isfile("StudentDetails\StudentDetails.csv")
    if exists1:
        df = pd.read_csv("StudentDetails\StudentDetails.csv")
    else:
        mess._show(title='Details Missing', message='Students details are missing, please check!')
        cam.release()
        cv2.destroyAllWindows()
        window.destroy()
    while flag:
        ret, im = cam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.2, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (225, 0, 0), 2)
            serial, conf = recognizer.predict(gray[y:y + h, x:x + w])
            if (conf < 50):
                ts = time.time()
                date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa = df.loc[df['SERIAL NO.'] == serial]['NAME'].values
                ID = df.loc[df['SERIAL NO.'] == serial]['ID'].values
                
                ID = str(ID)
                ID = ID[1:-1]
                bb = str(aa)
                bb = bb[2:-2]
                attendance = [str(ID), '', bb, '', str(date), '', str(timeStamp), '', 'in']
                flag = False

            else:
                Id = 'Unknown'
                bb = str(Id)
            cv2.putText(im, str(bb), (x, y + h), font, 1, (255, 255, 255), 2)
        cv2.imshow('Taking Attendance', im)
        
        if cv2.waitKey(1) == ord('q'):
            break
    ts = time.time()
    date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
    exists = os.path.isfile("Attendance\Attendance_" + date + ".csv")
    if exists:
        with open("Attendance\Attendance_" + date + ".csv", 'a+', encoding='utf-8') as csvFile1:
            writer = csv.writer(csvFile1)
            writer.writerow(attendance)
        csvFile1.close()
    else:
        with open("Attendance\Attendance_" + date + ".csv", 'a+', encoding='utf-8') as csvFile1:
            writer = csv.writer(csvFile1)
            writer.writerow(col_names)
            writer.writerow(attendance)
        csvFile1.close()
    with open("Attendance\Attendance_" + date + ".csv", 'r', encoding='utf-8') as csvFile1:
        reader1 = csv.reader(csvFile1)
        for lines in reader1:
            i = i + 1
            if (i > 1):
                if (i % 2 != 0):
                    iidd = str(lines[0]) + '  '
                    tv.insert('', 0, text=iidd, values=(str(lines[2]), str(lines[4]), str(lines[6]), str(lines[8])))
    csvFile1.close()
    cam.release()
    cv2.destroyAllWindows()
    
    
def Sign_out():
    assure_path_exists("Attendance/")
    assure_path_exists("StudentDetails/")
    
    flag = True
    """
    for k in tv.get_children():
        tv.delete(k)
    """
    msg = ''
    i = 0
    j = 0
    recognizer = cv2.face.LBPHFaceRecognizer_create()  # cv2.createLBPHFaceRecognizer()
    exists3 = os.path.isfile("TrainingImageLabel\Trainner.yml")
    if exists3:
        recognizer.read("TrainingImageLabel\Trainner.yml")
    else:
        mess._show(title='Data Missing', message='Please click on Save Profile to reset data!!')
        return
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath);

    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    col_names = ['Id', '', 'Name', '', 'Date', '', 'Time', '', 'State']
    exists1 = os.path.isfile("StudentDetails\StudentDetails.csv")
    if exists1:
        df = pd.read_csv("StudentDetails\StudentDetails.csv")
    else:
        mess._show(title='Details Missing', message='Students details are missing, please check!')
        cam.release()
        cv2.destroyAllWindows()
        window.destroy()
    while flag:
        ret, im = cam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.2, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x, y), (x + w, y + h), (225, 0, 0), 2)
            serial, conf = recognizer.predict(gray[y:y + h, x:x + w])
            if (conf < 50):
                ts = time.time()
                date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
                timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                aa = df.loc[df['SERIAL NO.'] == serial]['NAME'].values
                ID = df.loc[df['SERIAL NO.'] == serial]['ID'].values
                
                ID = str(ID)
                ID = ID[1:-1]
                bb = str(aa)
                bb = bb[2:-2]
                
                # 判斷 state 是不是 'in' or '' 
                temp_id = []
                with open("Attendance\Attendance_" + date + ".csv", 'r', encoding='utf-8') as csvFile1:
                    for line in csvFile1.readlines():
                        if line == '\n':
                            continue
                        else:
                            line = line.split(',')
                            line[8] = line[8].replace('\n', '')
                            temp_id.append(line[0])
                
                
                for idd in temp_id:
                    if ID == ( idd ):
                        tv.insert('', 0, text=idd, values=(bb, str(date), str(timeStamp), 'out'))
                        #tv.insert(str(ID), '', bb, '', str(date), '', str(timeStamp), '', 'fout')
                        data = pd.read_csv("Attendance\Attendance_" + date + ".csv", index_col = 'Id')
                        data.loc[ID, 'State'] = 'in and out'
                        data.to_csv(r"Attendance\Attendance_" + date + ".csv", index='Id', encoding='utf-8')
                        flag = False
                        break

            else:
                Id = 'Unknown'
                bb = str(Id)
            cv2.putText(im, str(bb), (x, y + h), font, 1, (255, 255, 255), 2)
        cv2.imshow('Taking Attendance', im)
        
        if cv2.waitKey(1) == ord('q'):
            break
    ts = time.time()
    date = datetime.datetime.fromtimestamp(ts).strftime('%d-%m-%Y')
    exists = os.path.isfile("Attendance\Attendance_" + date + ".csv")
    #tv.insert('', 0, text='test_str', values=(str(lines[2]), str(lines[4]), str(lines[6]), str(lines[8])))
    #csvFile1.close()
    cam.release()
    cv2.destroyAllWindows()    
    
######################################## USED STUFFS ############################################ 
global key
key = ''
# 日期
date = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d')

In [26]:
global txt, txt2, message1,message
def createNewWindow():
    frame2 = tk.Toplevel(frame1)
    frame2.geometry('500x450')
    lbl = tk.Label(frame2, text="輸入學號", width=23, height=1, fg="black", font=('Microsoft JhengHei', 17, ' bold '))
    lbl.place(x=80, y=55)

    global txt, txt2, message1,message
    txt = tk.Entry(frame2, width=45, fg="black", font=('times', 15, ' bold '))
    txt.place(x=30, y=88)

    lbl2 = tk.Label(frame2, text="輸入姓名", width=23, fg="black", font=('Microsoft JhengHei', 17, ' bold '))
    lbl2.place(x=80, y=140)

    txt2 = tk.Entry(frame2, width=45, fg="black", font=('times', 15, ' bold '))
    txt2.place(x=30, y=173)

    message1 = tk.Label(frame2, text="", fg="black", width=40, height=1, activebackground = "yellow", font=('times', 15, 'bold'))
    #message1.place(x=7, y=230)

    message = tk.Label(frame2, text="", fg="black", width=40, height=1, activebackground = "yellow", font=('times', 16, 'bold'))
    message.place(x=7, y=370)
    takeImg = tk.Button(frame2, text="拍照", command=TakeImages, fg="black", bg="yellow", width=36, height=1, activebackground = "white", font=('times', 15, 'bold'))
    takeImg.place(x=30, y=250)
    trainImg = tk.Button(frame2, text="儲存", command=TrainImages ,fg="black"  ,bg="yellow"  ,width=36  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
    trainImg.place(x=30, y=330)
    res=0
    exists = os.path.isfile('StudentDetails\StudentDetails.csv')
    if exists:
        with open("StudentDetails\StudentDetails.csv", 'r', encoding='utf-8') as csvFile1:
            reader1 = csv.reader(csvFile1)
            for l in reader1:
                res = res + 1
        res = (res // 2) - 1
        csvFile1.close()
    else:
        res = 0
    message.configure(text='總註冊人數  : ' + str(res))
######################################## GUI FRONT-END ###########################################
window = tk.Tk()
window.geometry('600x800')
window.resizable(0, 0)
window.title("點名系統")
#window.configure(background = '#262523')

frame1 = tk.Frame(window)
frame1.place(relx=0.11, rely=0.20)

#frame2 = tk.Frame(window)
#frame2.place(relx=0.51, rely=0.17, relwidth=0.38, relheight=0.80)

message3 = tk.Label(window, text = 'XXX點名系統', fg = "white", bg = '#262523', width=27, height = 1, font=('times', 29, 'bold'))
message3.place(x=0, y=10)

#frame3 = tk.Frame(window)
#frame3.place(relx=0.11, rely=0.09)

frame4 = tk.Frame(window)
frame4.place(relx=0.11, rely=0.09)

datef = tk.Label(frame4, text = date , fg="white", bg="#262523", width=27, height=1, font=('times', 22, 'bold'))
datef.grid()

clock = tk.Label(frame4, fg="white", bg="#262523", width=27, height=1, font=('times', 22, 'bold'))
clock.grid()
tick()



#head1 = tk.Label(frame1, text="                       已註冊                       ", fg="black", bg="#3ece48", font=('times', 17, 'bold'))
#head1.place(x=0,y=0)



lbl3 = tk.Label(frame1, text="出席狀況",width=10  ,fg="black"   ,height=1 ,font=('Microsoft JhengHei', 18))
lbl3.place(x=5, y=170)



################## TREEVIEW ATTENDANCE TABLE ####################

tv= ttk.Treeview(frame1, height = 18, columns = ('name', 'date', 'time', 'state'))
tv.column('#0',width=82)
tv.column('name',width=99)
tv.column('date',width=99)
tv.column('time',width=99)
tv.column('state',width=99)
tv.grid(row=2, column=0, padx=(0,0), pady=(200,0), columnspan=5)
tv.heading('#0', text ='學號')
tv.heading('name', text ='姓名')
tv.heading('date', text ='日期')
tv.heading('time', text ='時間')
tv.heading('state', text='狀態')

###################### SCROLLBAR ################################

scroll = ttk.Scrollbar(frame1, orient='vertical', command=tv.yview)
scroll.grid(row=2, column=5, padx=(0,100), pady=(150,0), sticky='ns')
tv.configure(yscrollcommand=scroll.set)

###################### BUTTONS ##################################

trackImg = tk.Button(frame1, text="簽到", command=TrackImages  ,fg="black"  ,bg="yellow"  ,width=35  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
trackImg.place(x=20,y=5)

trackImg = tk.Button(frame1, text="簽退", command=Sign_out  ,fg="black"  ,bg="yellow"  ,width=35  ,height=1, activebackground = "white" ,font=('times', 15, ' bold '))
trackImg.place(x=20,y=55)

head2 = tk.Button(frame1, text="註冊新用戶", fg="black", bg="#3ece48", font=('times', 15, 'bold'),width=35  ,height=1, command=createNewWindow)
head2.place(x=20,y=105)
##################### END ######################################
window.mainloop()
####################################################################################################

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
